#1 Introducción

En el siguiente cuaderno se realiza el algoritmo que obtiene la la suma de 2  matrices que guardan el resultado de la misma en una matriz de resultados (el orden de las matrices se ingresa por parametro)  aplicando paralelismo mediante Threads [4] y comunicando los mismos con el hilo principal mediante 2 colas y todo esto utilizando la CPU.

El algoritmo de este cuadernos se basa en, valga la redundancia, del algortimo para sumar 2 matrices proveniente del algebra lineal[3]

El objetivo es bajar los tiempos de ejecución repartiendo la carga del procesamiento de los datos en distintos hilos e implementando paralelismo para realizar la suma de 2  matrices, utilizando el lenguaje python [2] y la plataforma colab [1].


#2 Armado del ambiente

No son necesarios, ejecuciones previas del armado del ambiente.

#3 Desarrollo

In [19]:
#@title Parametros: { vertical-output: true, display-mode: "both" }
ordenMatriz =  1#@param {type:"number"}

from datetime import datetime
import threading
import numpy
import queue

tiempoTotal = datetime.now()

tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
try:
  cola1 = queue.Queue()
  cola2 = queue.Queue()

  def operacionesMatriz1(matriz,ordenMatriz):
    print(matriz)
    for i in range(0,ordenMatriz):
      for j in range(0,ordenMatriz):
        cola1.put(matriz[i][j])

  def operacionesMatriz2(matriz,ordenMatriz):
    print(matriz)
    for i in range(0,ordenMatriz):
      for j in range(0,ordenMatriz):
        cola2.put(matriz[i][j])

  matriz1 = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz1 = matriz1.astype(numpy.float32())
  matriz2 = numpy.random.random((ordenMatriz,ordenMatriz))
  matriz2 = matriz2.astype(numpy.float32())
  matrizR = numpy.empty_like(matriz1)

  print("Matrices a sumar: ")
  thread1 = threading.Thread(target=operacionesMatriz1,args=(matriz1,ordenMatriz,))
  thread1.start()
  
  thread2 = threading.Thread(target=operacionesMatriz2,args=(matriz2,ordenMatriz,))
  thread2.start()
  thread2.join()

  print("")
  for i in range(0,ordenMatriz):
    for j in range(0,ordenMatriz):
        numeroM1 = cola1.get()
        numeroM2 = cola2.get()
        matrizR[i][j] = (numeroM1+numeroM2)

  tiempoTotal = datetime.now() - tiempoTotal

  print("Matriz Resultado de la suma: ")
  print(matrizR)
  print("")
  print("Tiempo Total: ",tiempo_en_ms(tiempoTotal),"[ms]")

except ValueError:
  print("se ha ingresado un numero de orden menor a 0")
  print("ingrese un numero de orden de la matriz igual o mayor a 0")


Matrices a sumar: 
[[0.230688]]
[[0.5345379]]

Matriz Resultado de la suma: 
[[0.7652259]]

Tiempo Total:  7.635 [ms]



 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño(orden) de las matrices desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  cola1 = queue.Queue() cola2 = queue.Queue()| inicializo 2 colas
CPU      |  numpy.random.randn( ordenMatriz ) | Inicializa las matrices
CPU      |  thread1 = threading.Thread(target=operacionesMatriz1,args=(matriz1,ordenMatriz,))| inicializo thread1  
CPU      | thread1.start()        | ejecuta la funcion asociada el thread
CPU      | for                    | leo y pongo los valores de la matriz correspondiente en la cola correspondiente
CPU      |  thread2 = threading.Thread(target=operacionesMatriz1,args=(matriz2,ordenMatriz,))| inicializo thread2  
CPU      | thread2.start()        | ejecuta la funcion asociada el thread
CPU      | for                    | leo y pongo los valores de la matriz correspondiente en la cola correspondiente
CPU      | for                    | leo valores de ambas colas, realizo la suma y los guardo en la matriz de resultados en la posicion correspondiente
CPU      |  print()               | Informo los resultados.

#5 Conclusiones

En este ejercicio que se basa en el algoritmo de la suma de 2 matrices, utilice 2 Threads los cuales cada uno llama a la función que le corresponde y ejecuta esa función, cargando los valores de las matrices correspondientes en la cola correspondiente, estas colas las implemente para comunicar, los datos cargados de las matrices al hilo principal y que este último se encargue de hacer la suma, leyendo un dato de cada cola y sumando los mismos y almacenandolos en la matriz de resultados en la posición correspondiente.

Aunque intente dividir la carga del trabajo en diversos hilos, los tiempos de ejecución fueron elevados por la cantidad de bucles que hay en las funciones y en el hilo principal.

Por este motivo, si se quisiera mejorar la eficiencia del ejercicio seria optimo utilizar la GPU y usar el modulo CUDA, porque podriamos evitarnos todos estos bucles que mencione anteriormente, debido a la gran cantidad de hilos que nos ofrece CUDA  y poder así bajar el tiempo de ejecución


#6 Bibliografia

[1] MARKDOWN SYNTAX Colab:[PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [pagina colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Función de suma de matrices: [WEB](https://es.wikipedia.org/wiki/Matriz_(matem%C3%A1ticas)#Suma_o_adici%C3%B3n)

[4] Threads en python : [WEB](http://chuwiki.chuidiang.org/index.php?title=Hilos_en_python)
